# **This notebook helps to show how transfer learning can be implemented with VGG16 and other popular Pretrained Networks**

First make the neccesary imports. the Dataset used here for training was gotten from the 2018 Hackexpo organized by @AISaturdayLagos
https://www.kaggle.com/c/hackexpo2018-pre-competition

In [0]:
#Make the neccesary imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
from keras.utils import to_categorical as tc
from sklearn.preprocessing import StandardScaler



Using TensorFlow backend.


In [0]:
#Install kaggle and upload your kaggle API key in order to pull the competition dataor manually upload it
!pip install kaggle
from google.colab import files
files.upload()

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [0]:
!kaggle competitions download -c hackexpo2018

404 - Not Found


In [0]:
#Unzip the zip files to their respective folders
!unzip -q test.zip
!unzip -q train.zip

In [0]:
ls -lah

total 86M
drwxr-xr-x 1 root root 4.0K Jan 26 18:41 ./
drwxr-xr-x 1 root root 4.0K Jan 26 18:27 ../
drwxr-xr-x 1 root root 4.0K Jan  8 17:14 .config/
-rw-r--r-- 1 root root  43M Jan 26 18:38 hackexpo2018.zip
-rw-r--r-- 1 root root   66 Jan 26 18:28 kaggle.json
drwxr-xr-x 1 root root 4.0K Jan  8 17:15 sample_data/
---------- 1 root root  35K Nov 24 07:01 sample-submission.csv
drwxr-xr-x 2 root root  60K Nov 22 08:42 test/
---------- 1 root root 7.8M Nov 24 07:01 test.zip
drwxrwxr-x 6 root root 4.0K Nov 22 08:12 train/
---------- 1 root root  35M Nov 24 07:01 train.zip


## **Preparing Train Data**

In [0]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_FOLDER = "train"
TEST_FOLDER = "test"
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(rescale=1./255,
                          validation_split=0.15)
test_datagen = ImageDataGenerator(rescale=1./255)


train_data = train_datagen.flow_from_directory(TRAIN_FOLDER,target_size=(200,200),batch_size=BATCH_SIZE,class_mode="categorical", subset='training')
val_data = train_datagen.flow_from_directory(TRAIN_FOLDER,target_size=(200,200),batch_size=BATCH_SIZE,class_mode="categorical", subset='validation')


Found 3060 images belonging to 4 classes.
Found 540 images belonging to 4 classes.


## Preparing Test Data

In [0]:
!mkdir test/test
!mv  -v test/*.jpg test/test

In [0]:
datagen = ImageDataGenerator(rescale=1./255)
gen = datagen.flow_from_directory(r'test',target_size=(200, 200),shuffle=0, class_mode=None,batch_size=1)
filenames = gen.filenames
names=[]
for x in filenames:
    names.append(x.split('/')[1])
    

Found 800 images belonging to 1 classes.


In [0]:
"""import numpy as np
np.shape(gen)"""

'import numpy as np\nnp.shape(gen)'

Make imports for the CNNs

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier


In [0]:

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [0]:
import keras
from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


The same preprocessing has been applied to all models. the images are color images and have been reshaped to 200x200 for faster computinig

**I would be comparing a Neural Network of 9 Convolution Layers , VGG16, Xception, VGG19, ResNet50, InceptionV3, InceptionResNetV2, MobileNet,  DenseNet, NASNet, MobileNetV2**

**Each would be fed the same set of images and their performace evaluated**

**DEFINING THE MODELS**

9 LAYER CONV NETWORK

In [0]:
def conv_layer(x,filters,stride=1):
  x = Conv2D(filters,kernel_size=3,padding="same",strides=stride)(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  return x
  

def SimpleNet(input_size=(200,200,3),num_classes=4):
  
  inputs = Input(shape=input_size)
  
  outputs = conv_layer(inputs,16,stride=2)
  outputs = conv_layer(outputs,32)
  outputs = conv_layer(outputs,32)
  
  outputs = conv_layer(outputs,32,stride=2)
  outputs = conv_layer(outputs,64)
  outputs = conv_layer(outputs,64)
  
  outputs = conv_layer(outputs,64,stride=2)
  outputs = conv_layer(outputs,128)
  outputs = conv_layer(outputs,128)
  outputs = Dropout(0.5)(outputs)
  
  outputs = GlobalAvgPool2D()(outputs)
  outputs = Dense(num_classes,activation="softmax")(outputs)
  
  return Model(inputs=inputs,outputs=outputs)

model = SimpleNet()
model.compile(loss="categorical_crossentropy",optimizer=Adam(lr=0.001),metrics=["accuracy"])

  

In [0]:
print(applications)

In [0]:
from keras import applications
model_vgg19 = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_xception = applications.xception.Xception(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_vgg16 = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_resnet50 = applications.resnet50.ResNet50(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_inceptionv3 = applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_inceptionv2 = applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))


58892288/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


219062272/219055592 [==============================] - 4s 0us/step


In [0]:
model_densenet121 = applications.densenet.DenseNet121(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))

model_densenet169 = applications.densenet.DenseNet169(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_densenet201 = applications.densenet.DenseNet201(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))

model_nasnet_large = applications.nasnet.NASNetLarge(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))
model_nasnet_mobile = applications.nasnet.NASNetMobile(weights = "imagenet", include_top=False, input_shape = (200, 200, 3))

19996672/19993432 [==============================] - 1s 0us/step


In [0]:
model_nasnet_mobile

## All hidden layers of the pretrained models havebeen frozen and only the last layer retrained. The last layer is shown below and "atttached" to the "head" of the pretrained model

Store all models in a list for accesibilty

In [0]:
model_list = [model_vgg19,model_xception ,model_vgg16,model_resnet50 ,model_inceptionv3,model_inceptionv2,
              model_densenet121 ,model_densenet169,model_densenet201 ,model_nasnet_large,model_nasnet_mobile]

Create a dataframe to hold all the results of the training for analysis later

In [0]:
import pandas as pd
results_df = pd.DataFrame({'train_acc':[],'train_loss':[],'val_acc':[],'val_loss':[]})
results_df = []
results_df

[]

In [0]:
from keras import optimizers
from keras.callbacks import ModelCheckpoint

def trainer(model):
    x = model.output
    x = Flatten()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu")(x)
    predictions = Dense(4, activation="softmax")(x)
    # creating the final model 
    model_final = Model(input = model.input, output = predictions)
    # compile the model 
    model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
    #use checkpointer to keep track of the best model and save it
    checkpoint = ModelCheckpoint(filepath='model.weights.best.hdf5',  monitor="val_acc",verbose = 1, save_best_only=True)
    #Train the model for only 10 epochs
    results = model_final.fit_generator(train_data,steps_per_epoch=int(4000/BATCH_SIZE),epochs=10,validation_data=val_data,validation_steps=int(2000/BATCH_SIZE),callbacks=[checkpoint])
    
    return results.history

In [0]:
count = 0
for model in model_list:
    count = count + 1
    print("Training model " + str(count) + " of " + str(len(model_list)) + " models")
    remaining_res.append(trainer(model))

Training model 1 of 1 models


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


Epoch 1/10
125/125 [==============================] - 152s 1s/step - loss: 0.9674 - acc: 0.6032 - val_loss: 0.4793 - val_acc: 0.8511

Epoch 00001: val_acc improved from -inf to 0.85112, saving model to model.weights.best.hdf5
Epoch 2/10
125/125 [==============================] - 97s 776ms/step - loss: 0.5213 - acc: 0.8069 - val_loss: 0.3173 - val_acc: 0.8976

Epoch 00002: val_acc improved from 0.85112 to 0.89757, saving model to model.weights.best.hdf5
Epoch 3/10
125/125 [==============================] - 97s 773ms/step - loss: 0.3674 - acc: 0.8677 - val_loss: 0.2523 - val_acc: 0.9228

Epoch 00003: val_acc improved from 0.89757 to 0.92276, saving model to model.weights.best.hdf5
Epoch 4/10
125/125 [==============================] - 97s 775ms/step - loss: 0.2784 - acc: 0.9016 - val_loss: 0.2371 - val_acc: 0.9212

Epoch 00004: val_acc did not improve from 0.92276
Epoch 5/10
125/125 [==============================] - 97s 774ms/step - loss: 0.2165 - acc: 0.9246 - val_loss: 0.2226 - val_acc

In [0]:
results.history

{'acc': [0.7678034101111241, 0.8478370221327968],
 'loss': [0.6289029064305209, 0.4391732856901119],
 'val_acc': [0.8727180531010424, 0.8958333340602193],
 'val_loss': [0.37564086778047845, 0.2971737580601035]}

In [0]:
res = results_df

In [0]:
df = pd.DataFrame({'results':remaining_res})
df.head()

,results
0,"{'val_loss': [0.4793470764790124, 0.3173487670..."


In [0]:
df.to_csv('results4.csv')

In [0]:
!mkdir test/test
!mv  -v test/*.jpg test/test

test/


In [0]:
datagen = ImageDataGenerator(rescale=1./255)
gen = datagen.flow_from_directory(r'test',target_size=(200, 200),shuffle=0, class_mode=None,batch_size=1)
filenames = gen.filenames



Found 800 images belonging to 1 classes.


'test/00525fa2-c886-4f4d-8bf7-fbb72c6185f5.jpg'

['test', 'a21fb01b-8fd2-491b-81f7-6643e4eff147.jpg']

In [0]:
names=[]
for x in filenames:
    names.append(x.split('/')[1])
    

In [0]:
names[0]

In [0]:
import numpy as np
np.shape(gen)

In [0]:
prediction = model_final.predict(gen)

In [0]:
!unzip hackexpo2018.zip

Archive:  hackexpo2018.zip
  inflating: sample-submission.csv   
  inflating: test.zip                
  inflating: train.zip               


In [0]:
import pandas as pd
from ast import literal_eval
df = pd.DataFrame(columns=['train_acc','train_loss','val_acc','val_loss'])
df1 = pd.read_csv('result.csv')

In [0]:
train_acc, train_loss,val_acc,val_loss = ([] for _ in range(4))


In [0]:
for i in range(len(df1)):
    for a in df1.iloc[i,:]:
      for x in literal_eval(a.results[i])['train_acc']:
        train_acc.append(x)
      for x in literal_eval(a.results[i])['train_loss']:
        train_loss.append(x)
      for x in literal_eval(a.results[i])['val_acc']:
        val_acc.append(x)
      for x in literal_eval(a.results[i])['val_loss']:
        val_loss.append(x)

In [0]:
df1.iloc[1,:]

results    {'val_loss': [0.5413931961838179, 0.3603656972...
Name: 1, dtype: object

In [0]:
ls

result.csv  sample_data/


In [0]:
b['val_loss']

[0.251345473570224,
 0.24074802697613473,
 0.21983482024624823,
 0.2839752429752454,
 0.19585063858942045,
 0.2636131054714201,
 0.22482005804474892,
 0.22865902564329346,
 0.23329645960684298,
 0.26366004823305744]

In [0]:
df1.head(9)